# NVIDIA TAO DetectNet_v2 for Infineon PSOC EDGE Devices

Welcome to this comprehensive guide on training and optimizing neural networks for deployment on Infineon PSOC EDGE devices using NVIDIA's Train Adapt Optimize (TAO) Toolkit. This notebook will walk you through the complete workflow from training a custom object detection model to optimizing it for efficient deployment on resource-constrained edge devices.

## What is TAO Toolkit?

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. 

Train Adapt Optimize (TAO) Toolkit is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users own data. The TAO Toolkit provides a streamlined workflow to:

1. **Train**: Fine-tune pre-trained models with your own data
2. **Adapt**: Adapt models to your specific use-case
3. **Optimize**: Optimize models for efficient deployment on edge devices

<img align="center" src="https://d29g4g2dyqv443.cloudfront.net/sites/default/files/akamai/TAO/tlt-tao-toolkit-bring-your-own-model-diagram.png" width="1080">

## What is DetectNet-V2?

DetectNet_v2, also known as GridBox object detection, is a highly optimized CNN-based object detection model designed for efficient inference. It works by:

1. Dividing an input image into a uniform grid
2. Predicting four normalized bounding-box parameters (xc, yc, w, h) and confidence value per output class for each grid cell
3. Post-processing detections using clustering algorithms such as DBSCAN, NMS, or HYBRID (DBSCAN + NMS)

DetectNet_v2 is particularly well-suited for resource-constrained environments like the Infineon PSOC EDGE devices due to its computational efficiency while maintaining high accuracy.

### Sample output predictions from a trained DetectNet_v2 model

<img align="center" src="https://miro.medium.com/v2/resize:fit:720/0*YaQDIKR4gRbP2-by" width="960">

<img align="center" src="https://miro.medium.com/v2/resize:fit:720/0*eY1qluSyldYl9qDw" width="960">

## Workflow Overview and Learning Objectives

This notebook demonstrates how to leverage NVIDIA TAO to train and optimize a DetectNet_v2 model for deployment on Infineon PSOC EDGE devices. You will learn how to:

1. **Environment Setup**: Configure your environment for TAO toolkit
2. **Data Preparation**: Prepare your dataset in the required format for training
3. **Model Training**: Train a ResNet-18 based DetectNet_v2 model on the COCO dataset
4. **Model Evaluation**: Evaluate the trained model's performance
5. **Model Pruning**: Optimize the model size by pruning unnecessary weights
6. **Model Retraining**: Recover accuracy after pruning through retraining
7. **Quantization-Aware Training (QAT)**: Further optimize the model for deployment on edge devices
8. **Model Export**: Export the optimized model to ONNX format for deployment
9. **Infineon Toolchain Integration**: Convert the model for Infineon PSOC EDGE devices

By the end of this notebook, you will have a trained, pruned, quantized, and deployment-ready object detection model optimized for Infineon PSOC EDGE devices.

## 1. Environment Setup

Let's begin by setting up our environment variables and configuring the workspace. These variables will be used throughout the notebook to reference directories and files.

### 1.1 Setting Environment Variables

The following cell sets up environment variables that define:
- The number of GPUs to use for training
- Directory paths for experiments, data, and specifications

**Note**: Please make sure to remove any stray artifacts/files from previous experiments in `$USER_EXPERIMENT_DIR` or `$DATA_DOWNLOAD_DIR` as they may interfere with creating a training graph for a new experiment.

In [ ]:
# Setting up env variables for cleaner command line commands.
import os

%env NUM_GPUS=2
%env USER_EXPERIMENT_DIR=/workspace/tao-experiments/detectnet_v2
%env DATA_DOWNLOAD_DIR=/workspace/tao-experiments/data

# Set this path if you don't run the notebook from the samples directory.
# %env NOTEBOOK_ROOT=~/tao-samples/detectnet_v2

# Please define this local project directory that needs to be mapped to the TAO docker session.
# The dataset expected to be present in $LOCAL_PROJECT_DIR/data, while the results for the steps
# in this notebook will be stored at $LOCAL_PROJECT_DIR/detectnet_v2
# !PLEASE MAKE SURE TO UPDATE THIS PATH!.

os.environ["LOCAL_PROJECT_DIR"] = "/teamspace/studios/this_studio"

os.environ["LOCAL_DATA_DIR"] = os.path.join(
    os.getenv("LOCAL_PROJECT_DIR", os.getcwd()),
    "data"
)
os.environ["LOCAL_EXPERIMENT_DIR"] = os.path.join(
    os.getenv("LOCAL_PROJECT_DIR", os.getcwd()),
    "detectnet_v2"
)

# Make the experiment directory 
! mkdir -p $LOCAL_EXPERIMENT_DIR

# The sample spec files are present in the same path as the downloaded samples.
os.environ["LOCAL_SPECS_DIR"] = os.path.join(
    os.getenv("NOTEBOOK_ROOT", os.getcwd()),
    "specs"
)
%env SPECS_DIR=/workspace/tao-experiments/detectnet_v2/specs
CLEARML_LOGGED_IN = False
WANDB_LOGGED_IN = False

# Showing list of specification files.
!ls -rlt $LOCAL_SPECS_DIR

### 1.2 Mapping Directories to TAO Docker

The following cell maps our local directories to the TAO docker container. This allows TAO to access our data and save the results properly.

The mapping includes:
- The data directory containing our dataset
- The specs directory containing configuration files
- The experiment directory where outputs will be stored

This step is crucial for ensuring proper data access and result persistence between your local filesystem and the TAO docker environment.

In [ ]:
# Mapping up the local directories to the TAO docker.
import json
mounts_file = os.path.expanduser("~/.tao_mounts.json")

# Define the dictionary with the mapped drives
drive_map = {
    "Mounts": [
        # Mapping the data directory
        {
            "source": os.environ["LOCAL_PROJECT_DIR"],
            "destination": "/workspace/tao-experiments"
        },
        # Mapping the specs directory.
        {
            "source": os.environ["LOCAL_SPECS_DIR"],
            "destination": os.environ["SPECS_DIR"]
        },
        {
            "source": "/teamspace/studios/this_studio/data",
            "destination": "/workspace/tao-experiments/data"
        }
    ],
    "DockerOptions":{
        "user": f"{os.getuid()}:{os.getgid()}"
    }
}

if CLEARML_LOGGED_IN:
    if "Envs" not in drive_map.keys():
        drive_map["Envs"] = []
    drive_map["Envs"].extend([
        {
            "variable": "CLEARML_WEB_HOST",
            "value": os.getenv("CLEARML_WEB_HOST")
        },
        {
            "variable": "CLEARML_API_HOST",
            "value": os.getenv("CLEARML_API_HOST")
        },
        {
            "variable": "CLEARML_FILES_HOST",
            "value": os.getenv("CLEARML_FILES_HOST")
        },
        {
            "variable": "CLEARML_API_ACCESS_KEY",
            "value": os.getenv("CLEARML_API_ACCESS_KEY")
        },
        {
            "variable": "CLEARML_API_SECRET_KEY",
            "value": os.getenv("CLEARML_API_SECRET_KEY")
        },
    ])

if WANDB_LOGGED_IN:
    if "Envs" not in drive_map.keys():
        drive_map["Envs"] = []
    # Weights and biases currently requires access to the
    # /.config directory in the docker. Therefore, the docker
    # must be instantiated as root user. With the cells mentioned below
    # we will be deleting the cells that set user roles.
    if "user" in drive_map["DockerOptions"].keys():
        del(drive_map["DockerOptions"]["user"])
    drive_map["Envs"].extend([
        {
            "variable": "WANDB_API_KEY",
            "value": os.getenv("WANDB_API_KEY")
        }
    ])

# Writing the mounts file.
with open(mounts_file, "w") as mfile:
    json.dump(drive_map, mfile, indent=4)

!cat ~/.tao_mounts.json

### 1.3 Installing the TAO Launcher

The TAO launcher is a Python package that provides the interface to the TAO toolkit. It's distributed as a Python wheel on PyPI and can be installed with pip.

**Prerequisites**:
- Python >=3.7, <=3.10.x
- docker-ce > 19.03.5
- docker-API 1.40
- nvidia-container-toolkit > 1.3.0-1
- nvidia-container-runtime > 3.4.0-1
- nvidia-docker2 > 2.5.0-1
- nvidia-driver > 455+

After installation, we verify the TAO launcher version to ensure everything is set up correctly.

In [ ]:
# SKIP this step IF you have already installed the TAO launcher wheel.
!pip3 install nvidia-tao

In [ ]:
# View the versions of the TAO launcher
!tao info --verbose

## 2. Prepare dataset and pre-trained model <a class="anchor" id="head-2"></a>

## 2. Data Preparation

In this section, we'll prepare the COCO dataset for training our object detection model. The dataset must be properly structured and converted to TFRecords format for efficient training.

### 2.1 Downloading the Dataset

We'll download the COCO dataset, which contains images and annotations for common objects in context. This dataset is widely used for object detection tasks and provides a good baseline for training models that can later be fine-tuned for specific applications.

In [ ]:
# Create local dir
!mkdir -p $LOCAL_DATA_DIR
!mkdir -p $LOCAL_EXPERIMENT_DIR
# Download and preprocess data
!tao model detectnet_v2 run bash $SPECS_DIR/download_coco.sh $DATA_DOWNLOAD_DIR

### 2.2 Verifying the Downloaded Dataset

Let's verify that our dataset has been correctly downloaded by checking the number of training images, labels, and testing images. This step ensures we have all the necessary data before proceeding to model training.

In [ ]:
# verify
import os

DATA_DIR = os.environ.get('LOCAL_DATA_DIR')
num_training_images = len(os.listdir(os.path.join(DATA_DIR, "raw-data/train2017")))
num_val_images = len(os.listdir(os.path.join(DATA_DIR, "raw-data/val2017")))
print("Number of images in the train set. {}".format(num_training_images))
print("Number of images in the val set. {}".format(num_val_images))

### 2.3 Converting Dataset to TFRecords

For efficient training with NVIDIA TAO, we need to convert our dataset to TFRecords format. TFRecords is the native file format for TensorFlow, optimized for handling large datasets and enabling faster data loading during training.

The following steps involve:
1. Reviewing the TFRecords conversion specification file
2. Creating a directory for storing the TFRecords
3. Running the conversion process
4. Verifying the generated TFRecords

In [ ]:
print("TFrecords conversion spec file for kitti training")
!cat $LOCAL_SPECS_DIR/detectnet_v2_tfrecords_coco_trainval.txt

In [ ]:
# Creating a new directory for the output tfrecords dump.
print("Converting Tfrecords for kitti trainval dataset")
!mkdir -p $LOCAL_DATA_DIR/tfrecords
!tao model detectnet_v2 dataset_convert \
                  -d $SPECS_DIR/detectnet_v2_tfrecords_coco_trainval.txt \
                  -o $DATA_DOWNLOAD_DIR/tfrecords/coco_trainval/coco_trainval \
                  -r $USER_EXPERIMENT_DIR/

In [ ]:
!ls -rlt $LOCAL_DATA_DIR/tfrecords/coco_trainval/

In [ ]:
!tao model detectnet_v2 run bash $SPECS_DIR/check_data.sh

### 2.4 Downloading the Pre-trained Model

To leverage transfer learning, we'll download a pre-trained ResNet-18 model from NVIDIA's NGC registry. Using a pre-trained model significantly reduces training time and often leads to better performance, especially when training data is limited.

For DetectNet_v2, the input is expected to be 0-1 normalized with input channels in RGB order. Therefore, we use models from the `nvidia/tao/pretrained_detectnet_v2` repository which are specifically prepared for this purpose.

In [ ]:
# Installing NGC CLI on the local machine.
## Download and install
%env CLI=ngccli_cat_linux.zip
!mkdir -p $LOCAL_PROJECT_DIR/ngccli

# Remove any previously existing CLI installations
!rm -rf $LOCAL_PROJECT_DIR/ngccli/*
!wget "https://ngc.nvidia.com/downloads/$CLI" -P $LOCAL_PROJECT_DIR/ngccli
!unzip -u "$LOCAL_PROJECT_DIR/ngccli/$CLI" -d $LOCAL_PROJECT_DIR/ngccli/
!rm $LOCAL_PROJECT_DIR/ngccli/*.zip 
os.environ["PATH"]="{}/ngccli/ngc-cli:{}".format(os.getenv("LOCAL_PROJECT_DIR", ""), os.getenv("PATH", ""))

In [ ]:
# List models available in the model registry.
!ngc registry model list nvidia/tao/pretrained_detectnet_v2:*

In [ ]:
# Create the target destination to download the model.
!mkdir -p $LOCAL_EXPERIMENT_DIR/pretrained_resnet18/

In [ ]:
# Download the pretrained model from NGC
!ngc registry model download-version nvidia/tao/pretrained_detectnet_v2:resnet18 \
    --dest $LOCAL_EXPERIMENT_DIR/pretrained_resnet18

In [ ]:
!ls -rlt $LOCAL_EXPERIMENT_DIR/pretrained_resnet18/pretrained_detectnet_v2_vresnet18

## 3. Training Configuration

Now that we have our dataset and pre-trained model, we need to configure the training parameters. The training specification file defines various aspects of the training process, including:

- Dataset paths and configurations
- Pre-trained model path
- Augmentation parameters for data diversification
- Training hyperparameters (batch size, learning rate, epochs, etc.)
- Evaluation settings

Let's examine the training specification file to understand the configuration for our DetectNet_v2 model.

In [ ]:
!cat $LOCAL_SPECS_DIR/detectnet_v2_train_resnet18_coco.txt

## 4. Model Training

With our environment set up, data prepared, and training configuration defined, we can now train our DetectNet_v2 model. This process will fine-tune the pre-trained ResNet-18 model on our specific dataset.

**Important Notes**:
- Training may take several hours to complete depending on your GPU configuration
- DetectNet_v2 supports restart from checkpoint if training is interrupted
- When using multiple GPUs, you may need to adjust batch size and learning rate accordingly

First, we'll ensure the experiment directory is clean by removing any previous training artifacts.

In [22]:
!rm -rf $USER_EXPERIMENT_DIR/experiment_dir_unpruned

### 4.1 Running the Training Process

Now we'll start the training process using the TAO toolkit. The command below:
1. Specifies the model type (detectnet_v2)
2. Provides the training specification file
3. Sets the output directory for the trained model
4. Names the model for easier reference
5. Specifies the number of GPUs to use

In [ ]:
!tao model detectnet_v2 train -e $SPECS_DIR/detectnet_v2_train_resnet18_coco.txt \
                        -r $USER_EXPERIMENT_DIR/experiment_dir_unpruned \
                        -n resnet18_detector \
                        --gpus $NUM_GPUS

In [ ]:
print('Model for each epoch:')
print('---------------------')
!ls -lh $LOCAL_EXPERIMENT_DIR/experiment_dir_unpruned/weights

## 5. Model Evaluation

After training our model, we need to evaluate its performance to understand how well it performs on the validation dataset. This gives us a baseline metric before we proceed with optimization steps.

The evaluation process calculates key metrics like precision, recall, and mean Average Precision (mAP) to quantify the model's accuracy in detecting objects.

In [ ]:
!tao model detectnet_v2 evaluate -e $SPECS_DIR/detectnet_v2_train_resnet18_coco.txt \
                           -m $USER_EXPERIMENT_DIR/experiment_dir_unpruned/weights/resnet18_detector.hdf5

## 6. Model Pruning

Pruning is a model optimization technique that removes unnecessary weights from the neural network, resulting in a smaller model with faster inference times. This is particularly important for deployment on resource-constrained devices like Infineon PSOC EDGE.

### 6.1 Understanding Pruning Parameters

The key parameters for pruning are:

- **Pruning threshold (`-pth`)**: Controls the trade-off between model size and accuracy. Higher values result in smaller models but potentially lower accuracy.
- **Equalization layers (`-el`)**: Specifies which layers should be considered for pruning.

**Note**: The optimal pruning threshold depends on your specific dataset and requirements. A value of 0.01 is often a good starting point for DetectNet_v2 models, but you may need to experiment to find the best balance between model size and accuracy.

In [26]:
# Create an output directory if it doesn't exist.
!mkdir -p $LOCAL_EXPERIMENT_DIR/experiment_dir_pruned

### 6.2 Performing Initial Pruning

We'll perform an initial pruning step targeting specific layers of the ResNet-18 model with a pruning threshold of 0.9. This removes a significant portion of the model weights while preserving the most important ones for maintaining accuracy.

In [ ]:
!tao model detectnet_v2 prune \
                  -m $USER_EXPERIMENT_DIR/experiment_dir_unpruned/weights/resnet18_detector.hdf5 \
                  -o $USER_EXPERIMENT_DIR/experiment_dir_pruned/resnet18_nopool_bn_detectnet_v2_pruned.hdf5 \
                  -pth 0.9 \
                  -el block_1b_conv_1 block_1b_conv_2 \
                    block_1b_conv_shortcut block_1c_conv_1 block_1c_conv_2 \
                    block_1c_conv_shortcut block_2a_conv_1 block_2a_conv_2 \
                    block_2a_conv_shortcut block_2b_conv_1 block_2b_conv_2 \
                    block_2b_conv_shortcut block_2c_conv_1 block_2c_conv_2 \
                    block_2c_conv_shortcut block_2d_conv_1 block_2d_conv_2 \
                    block_2d_conv_shortcut block_3a_conv_1 block_3a_conv_2 \
                    block_3a_conv_shortcut block_3b_conv_1 block_3b_conv_2 \
                    block_3b_conv_shortcut block_3c_conv_1 block_3c_conv_2 \
                    block_3c_conv_shortcut block_3d_conv_1 block_3d_conv_2 \
                    block_3d_conv_shortcut block_3e_conv_1 block_3e_conv_2 \
                    block_3e_conv_shortcut block_3f_conv_1 block_3f_conv_2 \
                    block_3f_conv_shortcut  block_4a_conv_1 block_4b_conv_1 \
                    block_4c_conv_1 block_4a_conv_2 block_4b_conv_2 \
                    block_4c_conv_2 block_4a_conv_shortcut \
                    block_4b_conv_shortcut block_4c_conv_shortcut

In [ ]:
!ls -rlt $LOCAL_EXPERIMENT_DIR/experiment_dir_pruned/

### 6.3 Performing Additional Pruning

To further reduce the model size, we'll apply a second pruning step to the model, targeting additional layers with a pruning threshold of 0.8. This progressive pruning approach helps maintain a better balance between model size and accuracy.

In [ ]:
!tao model detectnet_v2 prune \
                  -m $USER_EXPERIMENT_DIR/experiment_dir_pruned/resnet18_nopool_bn_detectnet_v2_pruned.hdf5 \
                  -o $USER_EXPERIMENT_DIR/experiment_dir_pruned/resnet18_nopool_bn_detectnet_v2_pruned_final.hdf5 \
                  -pth 0.8 \
                  -el -el conv1 block_1a_conv_1 block_1a_conv_2 block_1a_conv_shortcut

In [ ]:
!ls -rlt $LOCAL_EXPERIMENT_DIR/experiment_dir_pruned/

## 7. Model Retraining

After pruning, the model typically experiences some accuracy degradation. To recover this lost accuracy, we need to retrain the pruned model. This retraining process preserves the smaller model size while improving its performance.

### 7.1 Configuring Retraining

For retraining, we need to:
1. Use the pruned model as the starting point
2. Set `load_graph` to `true` in the model configuration to load the pruned model's architecture
3. Adjust training parameters if necessary

In [ ]:
# Printing the retrain experiment file. 
# Note: We have updated the experiment file to include the 
# newly pruned model as a pretrained weights and, the
# load_graph option is set to true 
!cat $LOCAL_SPECS_DIR/detectnet_v2_retrain_resnet18_coco.txt

### 7.2 Running Retraining

Now we'll retrain the pruned model to recover accuracy. This process typically requires fewer epochs than the initial training since we're fine-tuning rather than training from scratch.

In [ ]:
# Retraining using the pruned model as pretrained weights 
!tao model detectnet_v2 train -e $SPECS_DIR/detectnet_v2_retrain_resnet18_coco.txt \
                        -r $USER_EXPERIMENT_DIR/experiment_dir_retrain \
                        -n resnet18_detector_pruned \
                        --gpus $NUM_GPUS

In [ ]:
# Listing the newly retrained model.
!ls -rlt $LOCAL_EXPERIMENT_DIR/experiment_dir_retrain/weights

## 8. Evaluating the Retrained Model

After retraining, we need to evaluate the pruned and retrained model to ensure it has recovered the accuracy lost during pruning. This step helps us validate that our optimized model maintains acceptable performance for our use case.

This section evaluates the pruned and retrained model, using the `evaluate` command.

In [ ]:
!tao model detectnet_v2 evaluate -e $SPECS_DIR/detectnet_v2_retrain_resnet18_coco.txt \
                           -m $USER_EXPERIMENT_DIR/experiment_dir_retrain/weights/resnet18_detector_pruned.hdf5

## 9. Visualizing Inferences

To get a qualitative understanding of our model's performance, we'll run inference on test images and visualize the detection results. This helps us identify any specific weaknesses or strengths in our model's detection capabilities.

### 9.1 Preparing Test Samples

First, we'll select a subset of images from our validation set to use for visualization:

In [3]:
!mkdir -p $LOCAL_DATA_DIR/test_samples
!cp $(ls $LOCAL_DATA_DIR/raw-data/val2017/* | head -n 20) $LOCAL_DATA_DIR/test_samples/

### 9.2 Running Inference

Now we'll run inference on our test samples using the trained model. The inference process will:
1. Process each image in the test directory
2. Generate bounding box predictions using our trained DetectNet_v2 model
3. Save annotated images and detection labels

In [ ]:
# Running inference for detection on n images
!tao model detectnet_v2 inference -e $SPECS_DIR/detectnet_v2_inference_coco.txt \
                            -r $USER_EXPERIMENT_DIR/tlt_infer_testing \
                            -i $DATA_DOWNLOAD_DIR/test_samples

The `inference` tool produces two outputs. 
1. Overlain images in `$USER_EXPERIMENT_DIR/tlt_infer_testing/images_annotated`
2. Frame by frame bbox labels in kitti format located in `$USER_EXPERIMENT_DIR/tlt_infer_testing/labels`

*Note: To run inferences for a single image, simply replace the path to the -i flag in `inference` command with the path to the image.*

### 9.3 Visualizing Results

The inference process produces two types of outputs:
1. Annotated images with bounding boxes in `$USER_EXPERIMENT_DIR/tlt_infer_testing/images_annotated`
2. Detection labels in KITTI format in `$USER_EXPERIMENT_DIR/tlt_infer_testing/labels`

Let's create a function to visualize these results in a grid:

In [ ]:
# Simple grid visualizer
!pip3 install "matplotlib>=3.3.3, <4.0"
%matplotlib inline
import matplotlib.pyplot as plt
import os
from math import ceil
valid_image_ext = ['.jpg', '.png', '.jpeg', '.ppm']

def visualize_images(image_dir, num_cols=4, num_images=10):
    output_path = os.path.join(os.environ['LOCAL_EXPERIMENT_DIR'], image_dir)
    num_rows = int(ceil(float(num_images) / float(num_cols)))
    f, axarr = plt.subplots(num_rows, num_cols, figsize=[80,30])
    f.tight_layout()
    a = [os.path.join(output_path, image) for image in os.listdir(output_path) 
         if os.path.splitext(image)[1].lower() in valid_image_ext]
    for idx, img_path in enumerate(a[:num_images]):
        col_id = idx % num_cols
        row_id = idx // num_cols
        img = plt.imread(img_path)
        axarr[row_id, col_id].imshow(img) 

Now let's display the first 12 annotated images to visually inspect our model's detection performance:

In [ ]:
# Visualizing the first 12 images.
OUTPUT_PATH = 'tlt_infer_testing/images_annotated' # relative path from $USER_EXPERIMENT_DIR.
COLS = 4 # number of columns in the visualizer grid.
IMAGES = 12 # number of images to visualize.

visualize_images(OUTPUT_PATH, num_cols=COLS, num_images=IMAGES)

## 10. Model Export

For deployment on Infineon PSOC EDGE devices, we need to export our trained and optimized model to ONNX format. ONNX (Open Neural Network Exchange) is an open format for representing deep learning models that enables interoperability between different frameworks.

In this step, we'll export our pruned and retrained model to ONNX format using the TensorFlow-to-ONNX converter.

In [ ]:
!mkdir -p $LOCAL_EXPERIMENT_DIR/experiment_dir_final
# Removing a pre-existing copy of the onnx if there has been any.
import os
output_file=os.path.join(os.environ['LOCAL_EXPERIMENT_DIR'],
                         "experiment_dir_final/resnet18_detector.onnx")
if os.path.exists(output_file):
    os.system("rm {}".format(output_file))
!tao model detectnet_v2 export \
                  -m $USER_EXPERIMENT_DIR/experiment_dir_retrain/weights/resnet18_detector_pruned.hdf5 \
                  -e $SPECS_DIR/detectnet_v2_retrain_resnet18_coco.txt \
                  -o $USER_EXPERIMENT_DIR/experiment_dir_final/resnet18_detector.onnx \
                  --onnx_route tf2onnx

## 11. Quantization-Aware Training (QAT)

Quantization is a technique to further reduce model size and improve inference speed by representing weights and activations with lower precision (e.g., INT8 instead of FP32). This is especially important for deployment on resource-constrained edge devices like Infineon PSOC EDGE.

While post-training quantization is simple, it often leads to accuracy degradation. Quantization-Aware Training (QAT) addresses this by simulating quantization effects during training, allowing the model to adapt to these effects and maintain accuracy.

### 11.1 Converting Pruned Model to QAT and Retraining

To enable QAT, we simply set the `enable_qat` parameter in the training configuration to `true`. This instructs the TAO toolkit to simulate quantization during the training process, resulting in a model that maintains higher accuracy when actually quantized for deployment.

Let's first examine our QAT-enabled training configuration:

In [ ]:
# Printing the retrain experiment file. 
# Note: We have updated the experiment file to convert the
# pretrained model to qat mode by setting the enable_qat
# parameter.
!cat $LOCAL_SPECS_DIR/detectnet_v2_retrain_resnet18_coco_qat.txt

### 11.2 Running QAT Retraining

Next, we'll run the retraining process with QAT enabled. This will simulate the effects of quantization during training, helping the model adapt to the reduced precision.

In [ ]:
!tao model detectnet_v2 train -e $SPECS_DIR/detectnet_v2_retrain_resnet18_coco_qat.txt \
                        -r $USER_EXPERIMENT_DIR/experiment_dir_retrain_qat \
                        -n resnet18_detector_pruned_qat \
                        --gpus $NUM_GPUS

In [ ]:
!ls -rlt $LOCAL_EXPERIMENT_DIR/experiment_dir_retrain_qat/weights

### 11.3 Evaluating the QAT Model

After QAT retraining, we need to evaluate the model's performance to ensure that the simulated quantization hasn't significantly degraded accuracy. Ideally, the mAP of the QAT model should be comparable to that of the pruned and retrained model without QAT.

In [ ]:
!tao model detectnet_v2 evaluate -e $SPECS_DIR/detectnet_v2_retrain_resnet18_kitti_qat.txt \
                           -m $USER_EXPERIMENT_DIR/experiment_dir_retrain_qat/weights/resnet18_detector_pruned_qat.hdf5 \
                           -f tlt

### 11.4 Exporting the QAT Model to ONNX

Now that we have a trained and evaluated QAT model, we need to export it to ONNX format for deployment. The exported model will maintain the quantization-aware properties, making it suitable for INT8 inference.

In [ ]:
!rm -rf $LOCAL_EXPERIMENT_DIR/experiment_dir_final/resnet18_detector_qat.onnx
!tao model detectnet_v2 export \
                  -m $USER_EXPERIMENT_DIR/experiment_dir_retrain_qat/weights/resnet18_detector_pruned_qat.hdf5 \
                  -o $USER_EXPERIMENT_DIR/experiment_dir_final/resnet18_detector_qat.onnx \
                  -e $SPECS_DIR/detectnet_v2_retrain_resnet18_coco_qat.txt \
                  --onnx_route tf2onnx

## 12. Infineon PSOC EDGE Integration

In this final section, we'll prepare our model for deployment on Infineon PSOC EDGE devices using the Infineon Toolchain. This involves installing the necessary dependencies and using the IFX Tooling to convert our ONNX model to a format compatible with Infineon's neural processing units.

### 12.1 Installing Required Dependencies

First, we need to install the necessary packages for the Infineon toolchain:

In [ ]:
!pip install openvino_dev openvino2tensorflow tensorflow==2.8 tensorflow_datasets ethos-u-vela onnx

### 12.2 Converting the Model for Infineon PSOC EDGE

Now we'll use the Infineon Toolchain (IFX Tooling) to convert our QAT-trained ONNX model to a format compatible with Infineon PSOC EDGE devices. This process involves:

1. Specifying the path to our QAT ONNX model
2. Configuring the target hardware (Ethos-U55 neural processing unit)
3. Setting system parameters specific to the PSOC EDGE platform
4. Running the conversion process to generate deployment-ready artifacts

The configuration specifically targets the Ethos-U55-128 accelerator with the PSE84_M55_U55_400MHz system configuration, which is optimized for Infineon PSOC EDGE devices.

In [ ]:
from ifx_tooling import run_ifx_tooling, ModelConversionError
import os
from pathlib import Path

qat_onnx_model_path = os.path.join(os.environ['LOCAL_EXPERIMENT_DIR'], "experiment_dir_final/resnet18_detector_qat.onnx")
ifx_tooling_output_path = os.path.join(os.environ['LOCAL_EXPERIMENT_DIR'], "ifx_tooling")

config = {
    'vela_accelerator': 'ethos-u55-128',
    'vela_system_config': 'PSE84_M55_U55_400MHz',
    'vela_memory_mode': 'Sram_Only',
    'compress_to_fp16': False,
    'vela_ini_file_path': os.path.join(os.environ['LOCAL_PROJECT_DIR'], "vela.ini")
}

try:
    output_paths = run_ifx_tooling(
        onnx_model_path=qat_onnx_model_path,
        input_shape=[1, 3, 240, 320],
        output_dir=ifx_tooling_output_path,
        config=config
    )
    print("Generated artifacts:", output_paths)
except ModelConversionError as e:
    print(f"Conversion failed: {e}")

## 13. Conclusion and Next Steps

Congratulations! You have successfully:

1. **Trained** a ResNet-18 based DetectNet_v2 model on the COCO dataset
2. **Evaluated** the model to establish a performance baseline
3. **Pruned** the model to reduce its size and computational requirements
4. **Retrained** the pruned model to recover accuracy
5. **Applied Quantization-Aware Training** to prepare the model for efficient INT8 inference
6. **Exported** the optimized model to ONNX format
7. **Converted** the model for deployment on Infineon PSOC EDGE devices using the Infineon Toolchain

### Next Steps

To deploy this model on your Infineon PSOC EDGE device:

1. Transfer the generated artifacts from the `ifx_tooling` directory to your development environment
2. Use the Infineon ModusToolbox™ to integrate the model into your application
3. Implement the pre-processing and post-processing logic to handle inputs and outputs
4. Test and validate the deployment on your target hardware

### Further Optimization Possibilities

If you need additional performance improvements:

1. **Data Augmentation**: Enhance the training dataset with more varied examples
2. **Hyperparameter Tuning**: Fine-tune learning rates, batch sizes, and other parameters
3. **Model Architecture**: Consider alternative model architectures like MobileNet for even more efficiency
4. **Custom Dataset**: Train on a dataset specific to your application domain

This workflow provides a strong foundation for developing and deploying efficient AI models on Infineon PSOC EDGE devices, enabling you to bring intelligence to the edge with optimized performance and resource usage.